Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [11]:
url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 1% change in download progress.
    """
    global last_percent_reported
    percent = int(count * blockSize * 100 / totalSize)

    if last_percent_reported != percent:
        if percent % 5 == 0:
            sys.stdout.write("%s%%" % percent)
            sys.stdout.flush()
        else:
            sys.stdout.write(".")
            sys.stdout.flush()
      
        last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    if force or not os.path.exists(filename):
        print('Attempting to download:', filename) 
        filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
        print('\nDownload Complete!')
        statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

UnboundLocalError: local variable 'statinfo' referenced before assignment

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [8]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz

    if os.path.isdir(root) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()

    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root)) if os.path.isdir(os.path.join(root, d))]
  
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders, one per class. Found %d instead.' % (num_classes, len(data_folders)))
  
    return data_folders
  
print('train_filename is ', train_filename)
print('test_filename is ', test_filename)
print('\n')

train_folders = maybe_extract(train_filename)
print(train_folders)
print('\n')

test_folders = maybe_extract(test_filename)
print(test_folders)

NameError: name 'train_filename' is not defined

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [4]:
import random
dir_name = 'notMNIST_large'
folders = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
for folder in folders:
    image_name = random.choice(os.listdir(dir_name + '/' + folder))
    image_path = dir_name + '/' + folder + '/' + image_name
    display(Image(filename=image_path))

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [5]:
image_folder = train_folders[0]
image_files = os.listdir(image_folder)
image_file = os.path.join(image_folder, image_files[0])
print(image_file)

print(ndimage.imread(image_file).astype(float).shape)
print(ndimage.imread(image_file).astype(float)[0])
print((ndimage.imread(image_file).astype(float)[0] - 125.25) / 255.0) 

NameError: name 'train_folders' is not defined

In [6]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
    """Load the data for a single letter label."""
    image_files = os.listdir(folder)
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size), dtype=np.float32)
    print(folder)
    num_images = 0
    for image in image_files:
        image_file = os.path.join(folder, image)
        try:
            image_data = (ndimage.imread(image_file).astype(float) - pixel_depth / 2) / pixel_depth

            if image_data.shape != (image_size, image_size):
                raise Exception('Unexpected image shape: %s' % str(image_data.shape))

            dataset[num_images, :, :] = image_data
            num_images = num_images + 1
        except IOError as e:
            print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
    dataset = dataset[0:num_images, :, :]
    if num_images < min_num_images:
        raise Exception('Many fewer images than expected: %d < %d' % (num_images, min_num_images))
    
    print('Full dataset tensor:', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))
    return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
        if os.path.exists(set_filename) and not force:
            # You may override by setting force=True.
            print('%s already present - Skipping pickling.' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_letter(folder, min_num_images_per_class)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)

    return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
print('train_datasets is ', train_datasets)
print('\n')

test_datasets = maybe_pickle(test_folders, 1800)
print('test_folders is ', test_datasets)

NameError: name 'train_folders' is not defined

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [7]:
pickle_file = train_datasets[0]
with open(pickle_file, 'rb') as f:
    image_set = pickle.load(f)
    print(image_set.shape)    
    print(len(image_set))

    image_index = np.random.randint(len(image_set))
    print(image_index)
    image_data = image_set[image_index, :, :]
    plt.figure()
    plt.imshow(image_data, interpolation='grey')
    plt.show()

NameError: name 'train_datasets' is not defined

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [9]:
len_dict = {}
for folder in folders:
    pickle_file = dir_name + '/' + folder + '.pickle'
    image_set = pickle.load(open(pickle_file, 'rb'))
    len_dict[folder] = len(image_set)
print(len_dict)

{'D': 52911, 'J': 52911, 'E': 52912, 'A': 52909, 'B': 52911, 'G': 52912, 'I': 52912, 'F': 52912, 'C': 52912, 'H': 52912}


Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
def make_arrays(nb_rows, img_size):
    if nb_rows:
        dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
        labels = np.ndarray(nb_rows, dtype=np.int32)
    else:
        dataset, labels = None, None
    return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
    num_classes = len(pickle_files)

    valid_dataset, valid_labels = make_arrays(valid_size, image_size)
    train_dataset, train_labels = make_arrays(train_size, image_size)

    vsize_per_class = valid_size // num_classes
    tsize_per_class = train_size // num_classes
    
    start_v, start_t = 0, 0
    end_v, end_t = vsize_per_class, tsize_per_class
    end_l = vsize_per_class + tsize_per_class

    for label, pickle_file in enumerate(pickle_files):       
        try:
            with open(pickle_file, 'rb') as f:
                letter_set = pickle.load(f)
                # let's shuffle the letters to have random validation and training set
                np.random.shuffle(letter_set)

                if valid_dataset is not None:
                    valid_letter = letter_set[:vsize_per_class, :, :]
                    
                    valid_dataset[start_v:end_v, :, :] = valid_letter
                    valid_labels[start_v:end_v] = label
                    
                    start_v += vsize_per_class
                    end_v += vsize_per_class

                train_letter = letter_set[vsize_per_class:end_l, :, :]
                
                train_dataset[start_t:end_t, :, :] = train_letter
                train_labels[start_t:end_t] = label
                
                start_t += tsize_per_class
                end_t += tsize_per_class
        except Exception as e:
            print('Unable to process data from', pickle_file, ':', e)
            raise
    
    return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [71]:
print(valid_labels.shape)
print(train_labels[:100])
print(np.random.permutation(valid_labels.shape[0]))

(10000,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[3799 2201 8050 ..., 8639 2611  765]


In [72]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

In [77]:
print(train_labels[:100])
print(np.unique(train_labels))
print(np.unique(test_labels))
print(np.bincount(train_labels))
print(np.bincount(test_labels))

[7 4 2 0 9 1 1 5 6 1 9 1 1 0 7 2 4 9 4 1 4 4 3 4 4 9 9 4 3 5 0 5 6 6 5 8 9
 0 8 7 0 8 4 8 7 9 9 4 2 2 3 6 6 0 9 9 2 1 1 7 8 7 8 3 8 0 4 6 3 8 8 9 1 4
 6 8 6 0 5 9 5 2 3 9 9 2 6 6 8 2 4 7 0 7 3 0 5 8 4 0]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]
[20000 20000 20000 20000 20000 20000 20000 20000 20000 20000]
[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]


Finally, let's save the data for later reuse:

In [78]:
pickle_file = 'notMNIST.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'valid_dataset': valid_dataset,
        'valid_labels': valid_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,
        }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [79]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800503


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [7]:
def reshape(arr):
    return arr.reshape(arr.shape[0], arr.shape[1] * arr.shape[2])

def unique_rows(a):
    a = np.ascontiguousarray(a) # Return a contiguous array in memory
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1])) 

pickle_file = pickle.load(open("notMNIST.pickle", "rb"))

print(pickle_file['train_dataset'].shape)
reshape_train_dateset = reshape(pickle_file['train_dataset'])
print(reshape_train_dateset.shape)

unique_td = unique_rows(reshape_train_dateset)
print(unique_td.shape)

duplicate_rows = len(pickle_file['train_dataset']) - len(unique_td)
print(duplicate_rows)

(200000, 28, 28)
(200000, 784)
(187380, 784)
12620


---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [8]:
train_y = pickle_file['train_labels']
train_x = reshape(pickle_file['train_dataset'])

valid_y = pickle_file['valid_labels']
valid_x = reshape(pickle_file['valid_dataset'])

test_y = pickle_file['test_labels']
test_x = reshape(pickle_file['test_dataset'])
del(pickle_file)

def random_data(dataset, labels, number):
  permutation = np.random.permutation(number)
  shuffled_dataset = dataset[permutation]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

model = LogisticRegression();
segements = [50, 100, 1000, 5000, 10000]
for segement in segements:
    train_data, labels_data = random_data(train_x, train_y, segement)
    model.fit(train_data, labels_data)
    
    predict_data_from_train, predict_labels_from_train = random_data(train_x, train_y, segement)
    predict_y_for_train = model.predict(predict_data_from_train)
    train_score = model.score(predict_data_from_train, predict_labels_from_train)
    classifier_rate_for_train = np.mean(predict_y_for_train.ravel() == predict_labels_from_train.ravel())
    print('classifier_rate_for_train is %f with %d training samples' % (classifier_rate_for_train, segement))
    print('the_score_for_train is %f with %d training samples' % (train_score, segement))
    
    predict_data_from_valid, predict_labels_from_valid = random_data(valid_x, valid_y, segement)
    predict_y_for_valid = model.predict(predict_data_from_valid)
    valid_score = model.score(predict_data_from_valid, predict_labels_from_valid)
    classifier_rate_for_valid = np.mean(predict_y_for_valid.ravel() == predict_labels_from_valid.ravel())
    print('classifier_rate_for_valid is %f with %d training samples' % (classifier_rate_for_valid, segement))
    print('the_score_for_valid is %f with %d training samples' % (valid_score, segement))
    
    
    predict_data_from_test, predict_labels_from_test = random_data(test_x, test_y, segement)
    predict_y_for_test = model.predict(predict_data_from_test)
    test_score = model.score(predict_data_from_test, predict_labels_from_test)
    classifier_rate_for_test = np.mean(predict_y_for_test.ravel() == predict_labels_from_test.ravel())
    print('classifier_rate_for_test is %f with %d training samples' % (classifier_rate_for_test, segement))
    print('the_score_for_test is %f with %d training samples' % (test_score, segement))
    print('\n')
    
    # model.fit(train_x[:segement], train_y[:segement])
    
    # predict_y_for_train = model.predict(train_x[:segement])
    # classifier_rate_for_train = np.mean(predict_y_for_train.ravel() == train_y[:segement].ravel())
    # print('classifier_rate_for_train is %f with %d training samples' % (classifier_rate_for_train, segement))
    
    # predict_y = model.predict(test_x[:segement])
    # classifier_rate_for_test = np.mean(predict_y.ravel() == test_y[:segement].ravel())
    # print('classifier_rate_for_test is %f with %d training samples' % (classifier_rate_for_test, segement))
    
    # valid_predict_y = model.predict(valid_x[:segement])
    # classifier_rate_for_valid = np.mean(valid_predict_y.ravel() == valid_y[:segement].ravel())
    # print('classifier_rate_for_valid is %f with %d training samples' % (classifier_rate_for_valid, segement))
    # print('\n')

classifier_rate_for_train is 1.000000 with 50 training samples
the_score_for_train is 1.000000 with 50 training samples
classifier_rate_for_valid is 0.560000 with 50 training samples
classifier_rate_for_test is 0.720000 with 50 training samples
the_score_for_valid is 0.720000 with 50 training samples


classifier_rate_for_train is 1.000000 with 100 training samples
the_score_for_train is 1.000000 with 100 training samples
classifier_rate_for_valid is 0.540000 with 100 training samples
classifier_rate_for_test is 0.690000 with 100 training samples
the_score_for_valid is 0.690000 with 100 training samples


classifier_rate_for_train is 0.997000 with 1000 training samples
the_score_for_train is 0.997000 with 1000 training samples
classifier_rate_for_valid is 0.744000 with 1000 training samples
classifier_rate_for_test is 0.815000 with 1000 training samples
the_score_for_valid is 0.815000 with 1000 training samples


classifier_rate_for_train is 0.943000 with 5000 training samples
the_scor

In [ ]:
import tensorflow as tf

mnist = {}

x = tf.placeholder("float", [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init = tf.initialize_all_variables()

session = tf.Session()
session.run(init)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_bath(100)
    session.run(train_step, feed_dict={x: batch_xs, y: batch_ys})


correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

print(session.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))